# Evaluation Code

In [ ]:
import pandas as pd
from visibility import Visibility
from glob import glob
import trimesh as tm
import pickle
import os
import numpy as np
from scipy.sparse import lil_matrix
from tqdm import tqdm

mesh_base_dir = '/home/motion/Surface_Aware_Disinfection/data/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val'
code_base_dir = os.getcwd()

def get_irradiance_matrix(vis_tester,sampling_places):
    total_faces = vis_tester.mesh.mesh.faces.shape[0]
    irradiance_matrix = lil_matrix((sampling_places.shape[0],total_faces))
    for i in range(sampling_places.shape[0]):
        _,irradiance = vis_tester.render(id0 =None,id1 = None,pos = sampling_places[i,:].tolist())
        irradiance = irradiance
        irradiance_matrix[i,np.where(irradiance > 0)] = irradiance[irradiance > 0]

    return irradiance_matrix.tocsr()

def evaluate_semantic_disinfection_performance(roadmap,robot):
    mesh_name = roadmap.split('/')[-4]
    # we then load the mesh original mesh and its ground truth:
    mesh_gt_file = os.path.join(mesh_base_dir, 'gt_{}.ply'.format(mesh_name))
    ground_truth_visibility = Visibility(mesh_gt_file,res = 512, useShader = True,createWnd = True)
    main_dir = os.path.dirname(roadmap)
    reachable_file = os.path.join(main_dir,'{}_reachable_330_divs.p'.format(robot))
    solutions_file = os.path.join(main_dir,'{}_solutions_330_divs.p'.format(robot))
    sampling_places_file = os.path.join(main_dir,'{}_sampling_places_330_divs.p'.format(robot))

    reachable = pickle.load(open(reachable_file,'rb'))
    solutions = np.array(pickle.load(open(solutions_file,'rb'))[0])
    sampling_places = pickle.load(open(sampling_places_file,'rb'))

    selected_points = sampling_places[reachable,:][solutions>0,:]
    irradiance_matrix = get_irradiance_matrix(ground_truth_visibility,selected_points)

    gt_mesh = ground_truth_visibility.mesh.mesh
    final_irradiances = (80*solutions[solutions>0].reshape(1,-1)@irradiance_matrix).flatten()
    areas = gt_mesh.area_faces
    colors = gt_mesh.visual.face_colors
    label = colors[:,0] >= 255
    disinfected = final_irradiances > 280
    total_ht_area = np.sum(areas[label])
    total_lt_area = np.sum(areas[~label])
    total_area = np.sum(areas)
    disinfected_ht_area = np.sum(areas[label][disinfected[label]])
    disinfected_lt_area = np.sum(areas[~label][disinfected[~label]])
    disinfected_area = np.sum(areas[disinfected])
    ht_area_fraction = total_ht_area/total_area
    return mesh_name,total_area,ht_area_fraction,disinfected_ht_area/total_ht_area,disinfected_area/total_area,disinfected_lt_area/total_lt_area


In [ ]:
robot = 'floatbot'

roadmaps = sorted(glob('./3D_results/Semantic/*/*/{}/{}_roadmap_330_divs.p'.format(robot,robot)))
mesh_names = []
experiments = []
durations = []
total_areas = []
ht_area_fractions = []
ht_disinfected_fraction = []
lt_disinfected_fraction = []
total_disinfected_fraction = []
robots = []
old_mesh = ''
for rm in tqdm(roadmaps):
    experiment = rm.split('/')[-3]
    duration = experiment.split('_')[-2]
    experiments.append(experiment)
    durations.append(duration)
    results = evaluate_semantic_disinfection_performance(rm,robot)
    mesh_names.append(results[0])
    total_areas.append(results[1])
    ht_area_fractions.append(results[2])
    ht_disinfected_fraction.append(results[3])
    total_disinfected_fraction.append(results[4])
    lt_disinfected_fraction.append(results[5])
    robots.append(robot)
results_df = pd.DataFrame({'robot':robot,'mesh_name':mesh_names,'experiment':experiments,'duration':durations,'room area':total_areas,
                          'ht_area_fraction':ht_area_fractions,'ht_disinfected_fraction':ht_disinfected_fraction,'lt_disinfected_fraction':lt_disinfected_fraction,'total_disinfected_fraction':total_disinfected_fraction})

In [ ]:
results_df.to_csv('./Semantic_results_large_rooms_2.csv',sep = '|')

In [ ]:
results_df

In [ ]:
results_df.loc[results_df.mesh_name == 'scene0011_00']

In [ ]:
### import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd


def add_agnostic_fractions_column(this_df):

    agnostic_fractions = []
    for i in range(this_df.shape[0]):
        row = this_df.iloc[i,:]
        duration = row.duration
        mesh_name = row.mesh_name
        agnostic_row = this_df[np.logical_and(this_df.experiment_names.str.startswith('surface'),np.logical_and(this_df.mesh_name ==mesh_name,this_df.duration == duration))]
        if(agnostic_row.empty):
            agnostic_fractions.append(None)
        else:
            agnostic_fractions.append(agnostic_row.ht_disinfected_fraction.values[0])
    this_df['agnostic_ht_fraction'] = agnostic_fractions
    return this_df


df = pd.read_csv('././Semantic_results_large_rooms.csv',sep = '|')

tmp = df.experiment.str.split('_',expand = True)
names = tmp.loc[:,0] + '_' + tmp.loc[:,1]
names[names.str.startswith('hard')] += '_' + tmp.loc[names.str.startswith('hard'),2]
df['experiment_names'] = names

df = add_agnostic_fractions_column(df)

# Uplift Plots

In [ ]:
df2 = df[df.duration.str.isnumeric()]
max_disinf = df.groupby(['mesh_name']).ht_disinfected_fraction.max()
df2.duration = df2.duration.astype(float)
for tmp in df2.groupby('mesh_name'):
    room_name = tmp[0]
    max_disinfection = max_disinf[room_name]
    title = 'HT-Uplift from Surface Agnostic {}'.format(tmp[0])
    plt.title(title)
    tmp = tmp[1]
    
    for tmp2 in tmp.groupby('experiment_names'):
        name = tmp2[0]
        tmp2 = tmp2[1]
        plt.scatter(y = ((tmp2.ht_disinfected_fraction-tmp2.agnostic_ht_fraction)/max_disinfection),x = tmp2.duration,label = name)
    plt.legend(loc = 4)
    plt.xlabel('ETime Budget (min)')
    plt.ylabel('HT fraction uplift from agnostic (n.d.)')
    plt.ylim([0.0,0.3])
    plt.savefig('./media/semantic_results/{}.jpg'.format(title))
    plt.show()

# HT_fraction plots

In [ ]:
df2 = df[df.duration.str.isnumeric()]
max_disinf = df.groupby(['mesh_name']).ht_disinfected_fraction.max()
df2.duration = df2.duration.astype(float)
for tmp in df2.groupby('mesh_name'):
    room_name = tmp[0]
    max_disinfection = max_disinf[room_name]
    title = 'Disinfected HT Fraction vs Time Budget {}'.format(tmp[0])
    plt.title(title)
    tmp = tmp[1]
    
    for tmp2 in tmp.groupby('experiment_names'):
        name = tmp2[0]
        tmp2 = tmp2[1]
        plt.scatter(y = ((tmp2.ht_disinfected_fraction)/max_disinfection),x = tmp2.duration,label = name)
    plt.legend(loc = 4)
    plt.xlabel('Time Budget(min)')
    plt.ylabel('HT disinfection fraction (n.d.)')
    plt.ylim([0.0,1.2])
    plt.savefig('./media/semantic_results/{}.jpg'.format(title))
    plt.show()

# Total Disinfection Fractions

In [ ]:
df2 = df[df.duration.str.isnumeric()]
max_disinf = df.groupby(['mesh_name']).ht_disinfected_fraction.max()
df2.duration = df2.duration.astype(float)
for tmp in df2.groupby('mesh_name'):
    room_name = tmp[0]
    max_disinfection = max_disinf[room_name]
    title = 'Disinfected Total Fraction vs Time Budget {}'.format(tmp[0])
    plt.title(title)
    tmp = tmp[1]
    
    for tmp2 in tmp.groupby('experiment_names'):
        name = tmp2[0]
        tmp2 = tmp2[1]
        plt.scatter(y = ((tmp2.total_disinfected_fraction)/max_disinfection),x = tmp2.duration,label = name)
    plt.legend(loc = 4)
    plt.xlabel('Time Budget(min)')
    plt.ylabel('total disinfection fraction (n.d.)')
    plt.ylim([0.0,1.2])
    plt.savefig('./media/semantic_results/{}.jpg'.format(title))
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('./Semantic_results.csv',sep = '|')

tmp = df.experiment.str.split('_',expand = True)
names = tmp.loc[:,0] + '_' + tmp.loc[:,1]
names[names.str.startswith('hard')] += '_' + tmp.loc[names.str.startswith('hard'),2]
df['experiment_names'] = names

df2 = df[df.duration < 100]
max_disinf = df2.groupby(['mesh_name']).total_disinfected_fraction.max()

for tmp in df2.groupby('mesh_name'):
    room_name = tmp[0]
    max_disinfection = max_disinf[room_name]
    title = 'Normalized GT disinfected fraction {}'.format(tmp[0])
    plt.title(title)
    tmp = tmp[1]
    for tmp2 in tmp.groupby('experiment_names'):
        name = tmp2[0]
        tmp2 = tmp2[1]
        plt.scatter(y = (tmp2.ht_disinfected_fraction/max_disinfection),x = tmp2.duration,label = name)
    plt.legend()
    plt.xlabel('Experiment Duration (min)')
    plt.ylabel('total_fraction \nasymptotic (n.d.)')
    plt.ylim([0,1.2])
    plt.savefig('./media/semantic_results/{}.pdf'.format(title))
    plt.show()

In [ ]:

for tmp in df2.groupby('mesh_name'):
    room_name = tmp[0]
    max_disinfection = max_disinf[room_name]
    title = 'HT uplift vs experiment duration {}'.format(tmp[0])
    plt.title(title)
    tmp = tmp[1]
    for tmp2 in tmp.groupby('experiment_names'):
        name = tmp2[0]
        tmp2 = tmp2[1]
        plt.scatter(y = (tmp2.ht_disinfected_fraction/max_disinfection-tmp2.total_disinfected_fraction/max_disinfection),x = tmp2.duration,label = name)
    plt.legend()
    plt.xlabel('Experiment Duration (min)')
    plt.ylabel('ht_fraction - total_fraction \nasymptotic (n.d.)')
    plt.ylim([-0.5,0.5])
    plt.savefig('./media/semantic_results/{}.pdf'.format(title))
    plt.show()

# Plotting Code

In [ ]:
from glob import glob 
import pandas as pd
from matplotlib import pyplot as plt

all_meshes = glob('/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/*.ply')

meshes_series = pd.Series(all_meshes)
estimated_mask = ~meshes_series.str.split('/',expand = True).iloc[:,-1].str.startswith('gt')
# we now get the number of done meshes

# done_files = glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/{}/armbot/*'.format(experiment))
# if(done_files):
#     done_meshes = pd.Series(done_files).str.split('/',expand = True).iloc[:,-4].tolist()
# else:
#     done_meshes = []

estimated_meshes = sorted(meshes_series[estimated_mask].tolist())

# Plotting Code

In [ ]:
from glob import glob 
import pandas as pd
from matplotlib import pyplot as plt

all_meshes = glob('/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/*.ply')

meshes_series = pd.Series(all_meshes)
estimated_mask = ~meshes_series.str.split('/',expand = True).iloc[:,-1].str.startswith('gt')
# we now get the number of done meshes

# done_files = glob('/home/motion/Optimized-UV-Disinfection/3D_results/Semantic/*/{}/armbot/*'.format(experiment))
# if(done_files):
#     done_meshes = pd.Series(done_files).str.split('/',expand = True).iloc[:,-4].tolist()
# else:
#     done_meshes = []

estimated_meshes = sorted(meshes_series[estimated_mask].tolist())

In [ ]:
a = ['this is bullshit {}','this is super bullshit {}']
b = a[0].format(12)
a

In [ ]:
import pandas as pd
df = pd.read_csv('./Semantic_results_shorter_time.csv',sep = '|')

In [ ]:
tmp = df.experiment.str.split('_',expand = True)
names = tmp.loc[:,0] + '_' + tmp.loc[:,1]
names[names.str.startswith('hard')] += '_' + tmp.loc[names.str.startswith('hard'),2]
df['experiment_names'] = names

df2 = df[df.duration < 100]
max_disinf = df2.groupby(['mesh_name']).total_disinfected_fraction.max()

In [ ]:
for tmp in df2.groupby('mesh_name'):
    room_name = tmp[0]
    max_disinfection = max_disinf[room_name]
    title = 'HT uplift vs experiment duration {}'.format(tmp[0])
    plt.title(title)
    tmp = tmp[1]
    for tmp2 in tmp.groupby('experiment_names'):
        name = tmp2[0]
        tmp2 = tmp2[1]
        plt.scatter(y = (tmp2.ht_disinfected_fraction/max_disinfection-tmp2.total_disinfected_fraction/max_disinfection),x = tmp2.duration,label = name)
    plt.legend()
    plt.xlabel('Experiment Duration (min)')
    plt.ylabel('ht_fraction - total_fraction \nasymptotic (n.d.)')
    plt.ylim([-0.1,0.1])
    plt.savefig('./media/semantic_results/{}.pdf'.format(title))
    plt.show()

# Mesh ordering

In [ ]:
from glob import glob
import trimesh as tm
from tqdm import tqdm
import pandas as pd
from alphashape import alphashape
import shapely
from joblib import Parallel,delayed
def get_maximum_connected_component_mesh(mesh_file):
    full_mesh = tm.exchange.load.load(mesh_file)
    # full_mesh.show()
    conn_components = tm.graph.split(full_mesh, only_watertight = False)
    max_faces = 0
    for component in conn_components:
        if(component.faces.shape[0]>max_faces):
            max_faces = component.faces.shape[0]
            maximum_connected_component  = component
    return maximum_connected_component

def get_floorplan_area(mesh_file):
    mesh = get_maximum_connected_component_mesh(mesh_file)
    projection = mesh.vertices[mesh.vertices[:,2]<0.1,:2]
    alpha_shape = alphashape(projection,10)
    # if the alpha_shape is a multipoligon, we get its component with largest area
    if(type(alpha_shape) == shapely.geometry.multipolygon.MultiPolygon):
        max_area = 0
        for shape in alpha_shape:
            if(shape.area>max_area):
                max_area = shape.area
                final_shape = shape
        alpha_shape = final_shape
    bounds = alpha_shape.bounds
    return alpha_shape,alpha_shape.area,mesh

In [ ]:
a,aa,mesh = get_floorplan_area('/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/scene0187_00.ply')

In [ ]:
a

In [ ]:
mesh.show()

In [ ]:
all_meshes = sorted(glob('/home/motion//Optimized-UV-Disinfection/data/environment_meshes/aug_10_entire_val_ade20kmodel_vanilla_prob_weighting/estimated_scannet_val/*.ply'))
areas = []
for mesh in tqdm(all_meshes):
    areas.append(get_floorplan_area(mesh))
# result = Parallel(n_jobs = 2,verbose = 100)(delayed(get_floorplan_area)(mesh) for mesh in all_meshes)

In [ ]:
areas_df = pd.DataFrame({"mesh_file":all_meshes,'floorplan_area':areas})

In [ ]:
sorted_areas = areas_df.sort_values(by = 'floorplan_area',ascending = False).reset_index(drop = True)

In [ ]:
large_areas = sorted_areas.loc[sorted_areas.area>100,:]

In [ ]:
b = tm.exchange.load.load('/home/motion/Surface_Aware_Disinfection/data/environment_meshes/remeshed_hospital_room_full_35k.obj')

In [ ]:
sorted_areas.loc[sorted_areas.floorplan_area > 36,:]

In [ ]:
sorted_areas['mesh_names'] = sorted_areas.mesh_file.str.split('/',expand = True).iloc[:,-1]

In [ ]:
sorted_areas.to_csv('meshes_by_floorplan_area.csv', sep = '|', index = False)

In [ ]:
import pandas as pd
a = pd.read_csv('meshes_by_floorplan_area.csv', sep = '|')

b = a.loc[~a.mesh_names.str.startswith('gt'),:]

In [ ]:
b = a.loc[~a.mesh_names.str.startswith('gt'),:]
len(b.loc[b.floorplan_area > 36,'mesh_file'].tolist())

# seeing failed meshes

In [ ]:
with open('./failed_meshes_floatbot.txt','r') as f:
    k = f.read()
import pandas as pd


In [ ]:
pd.Series(k.split('\n')).value_counts()

In [ ]:
df[~df.duration.str.isnumeric()].shape

In [ ]:
from glob import glob

In [ ]:
rm = pd.Series(glob('/home/motion/Surface_Aware_Disinfection/3D_results/Semantic/*/*/floatbot/floatbot_roadmap_330_divs.p'))

In [ ]:
'hard_cutoff_50_15_minutes' not in rm.str.split('/',expand = True).iloc[:,7].values 